In [2]:
import os
import glob
import pandas as pd

## Ensure that checkpoints exist

In [3]:
checkpoint_save_dir = "../data/raw/checkpoints"
merged_save_dir = "../data/raw"

if not os.path.exists(checkpoint_save_dir):
    print("Checkpoint directory does not exist. RUn the 01_queue-times-scrape.py script first.")

In [6]:
folders = [os.path.basename(folder) for folder in glob.glob(checkpoint_save_dir + "/*")]

print(f"Found {len(folders)} folders: {folders}")

Found 2 folders: ['309', '51']


## Merge all scraped checkpoint files and save to disk

In [25]:
def merge_files_in_dir(dir_name):
  ride_files = glob.glob(os.path.join(checkpoint_save_dir, dir_name, "ride_queue_times_checkpoint_*.parquet"))
  weather_files = glob.glob(os.path.join(checkpoint_save_dir, dir_name, "weather_data_checkpoint_*.parquet"))
  save_dir = os.path.join(merged_save_dir, dir_name)
  os.makedirs(save_dir, exist_ok=True)

  print(f"Found {len(ride_files)} ride checkpoints and {len(weather_files)} weather checkpoints.")

  if ride_files:
      ride_dfs = [pd.read_parquet(file) for file in ride_files]

      merged_rides = pd.concat([df for df in ride_dfs if not df.empty], ignore_index=True)
      merged_rides.to_parquet(os.path.join(merged_save_dir, dir_name, "merged_rides.parquet"), engine="pyarrow")
      print(f"Merged {len(ride_files)} ride checkpoint files with {len(merged_rides)} entries into 'merged_rides.parquet'")
  else:
      print("No ride checkpoint files found.")

  if weather_files:
      weather_dfs = [pd.read_parquet(file) for file in weather_files]

      merged_weather = pd.concat([df for df in weather_dfs if not df.empty], ignore_index=True)
      merged_weather.to_parquet(os.path.join(merged_save_dir, dir_name, "merged_weather.parquet"), engine="pyarrow")
      print(f"Merged {len(weather_files)} weather checkpoint files with {len(merged_weather)} entries into 'merged_weather.parquet'")
  else:
      print("No weather checkpoint files found.")

In [26]:
for folder in folders:
    merge_files_in_dir(folder)

Found 50 ride checkpoints and 50 weather checkpoints.
Merged 50 ride checkpoint files with 2215265 entries into 'merged_rides.parquet'
Merged 50 weather checkpoint files with 18191 entries into 'merged_weather.parquet'
Found 75 ride checkpoints and 75 weather checkpoints.
Merged 75 ride checkpoint files with 6268117 entries into 'merged_rides.parquet'
Merged 75 weather checkpoint files with 19943 entries into 'merged_weather.parquet'


/tmp/ipykernel_647816/1473639731.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_weather = pd.concat([df for df in weather_dfs if not df.empty], ignore_index=True)
